In [1]:
import sys
print(sys.version)

3.8.10 (default, Mar 15 2022, 12:22:08) 
[GCC 9.4.0]


In [2]:
import pandas as pd

In [3]:
from wefe.query import Query
from wefe.metrics import WEAT, RNSB, RND, MAC, ECT
from wefe.word_embedding_model import WordEmbeddingModel
from wefe.datasets.datasets import load_weat, fetch_eds, fetch_debias_multiclass, fetch_debiaswe, load_bingliu
from wefe.utils import run_queries, plot_queries_results, create_ranking, plot_ranking, plot_ranking_correlations
from plotly.subplots import make_subplots
import gensim

In [4]:
data_folder = "../../Data/"
embeddings_folder = data_folder+"/word embeddings/"

In [5]:
def get_google_news_embeddings(filename,binary=False):
    w2v_model = gensim.models.KeyedVectors.load_word2vec_format(filename, binary=binary, no_header=True)
    return w2v_model


In [6]:
fasttext_wiki_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"FastText/wiki-news-300d-1M.vec")

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings/FastText/wiki-news-300d-1M.vec
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings/FastText/wiki-news-300d-1M.vec', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (999994, 300) matrix of type float32 from ../../Data//word embeddings/FastText/wiki-news-300d-1M.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T10:55:30.194070', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-glibc2.29', 'event': 'load_word2vec_format'}


In [7]:
fasttext_wiki_subwords_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"FastText/wiki-news-300d-1M-subword.vec")

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings/FastText/wiki-news-300d-1M-subword.vec
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings/FastText/wiki-news-300d-1M-subword.vec', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (999994, 300) matrix of type float32 from ../../Data//word embeddings/FastText/wiki-news-300d-1M-subword.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T10:56:52.165239', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-glibc2.29', 'event': 'load_word2vec_format'}


In [8]:
fasttext_cc_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"FastText/crawl-300d-2M.vec")

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings/FastText/crawl-300d-2M.vec
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings/FastText/crawl-300d-2M.vec', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (1999995, 300) matrix of type float32 from ../../Data//word embeddings/FastText/crawl-300d-2M.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T10:59:19.959181', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-glibc2.29', 'event': 'load_word2vec_format'}


In [9]:
fasttext_cc_subwords_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"FastText/crawl-300d-2M-subword.vec")

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings/FastText/crawl-300d-2M-subword.vec
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings/FastText/crawl-300d-2M-subword.vec', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (2000000, 300) matrix of type float32 from ../../Data//word embeddings/FastText/crawl-300d-2M-subword.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T11:01:56.062477', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-glibc2.29', 'event': 'load_word2vec_format'}


In [10]:
debiased_w2v_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"/debiased_word_embedding/GoogleNews-vectors-negative300-hard-debiased.bin",binary=True)

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings//debiased_word_embedding/GoogleNews-vectors-negative300-hard-debiased.bin
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings//debiased_word_embedding/GoogleNews-vectors-negative300-hard-debiased.bin', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from ../../Data//word embeddings//debiased_word_embedding/GoogleNews-vectors-negative300-hard-debiased.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2022-04-19T11:02:20.101948', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-gli

In [11]:
# load the weat word sets
WEAT_wordsets = load_weat()
RND_wordsets = fetch_eds()
sentiments_wordsets = load_bingliu()
debias_multiclass_wordsets = fetch_debias_multiclass()

#### WEAT Effect Size: represents a normalized measure that quantifies how far apart the two distributions of association between targets and attributes are.

#### WEAT score:  Its objective is to quantify the strength of association of both pair of sets through a permutation test. The score being positive and higher than one indicates that word2vec exhibits a moderately strong relationship between men's names and careers and women's names and family.

In [12]:
profane_words = open(data_folder+"swear_words.txt").read().splitlines()

In [13]:
harmless_minority_words_clean = ["woman", "female", "girl","wife","sister","daughter","mother"]
harmless_majority_words_clean = ["man","male","boy","husband","brother","son","father"]


#### running multiple queries and multiple word embeddings

In [14]:
fasttext_wiki_model = WordEmbeddingModel(fasttext_wiki_model, 'fasttext_wiki')
fasttext_wiki_subwords_model = WordEmbeddingModel(fasttext_wiki_subwords_model, 'fasttext_wiki_subwords')
fasttext_cc_model = WordEmbeddingModel(fasttext_cc_model, 'fasttext_cc')
fasttext_cc_subwords_model = WordEmbeddingModel(fasttext_cc_subwords_model, 'fasttext_cc_subwords')
debiased_w2v_model = WordEmbeddingModel(debiased_w2v_model, 'debiased_w2v')

In [15]:
models = [fasttext_wiki_model, fasttext_wiki_subwords_model, fasttext_cc_model, fasttext_cc_subwords_model, debiased_w2v_model]

## Query ranking

In [16]:
#query1 = Query([harmless_minority_words_clean, harmless_majority_words_clean],
   #                    [WEAT_wordsets['unpleasant_5'], WEAT_wordsets['pleasant_5']])

query2 = Query([harmless_minority_words_clean, harmless_majority_words_clean],
                       [profane_words, sentiments_wordsets['positive_words']])

gender_queries = [query2]

In [17]:
# Run the queries WEAT
WEAT_gender_results = run_queries(
    WEAT,
    gender_queries,
    models,
    aggregate_results=True,
    return_only_aggregation=True,
    lost_vocabulary_threshold = 0.6,
    queries_set_name='SOS Queries')

# Run the queries with WEAT Effect Size

WEAT_EZ_gender_results = run_queries(WEAT,
                                     gender_queries,
                                     models,
                                     metric_params={
                                        
                                         'return_effect_size': True
                                     },
                                     aggregate_results=True,
                                     return_only_aggregation=True,
                                     lost_vocabulary_threshold = 0.6,
                                     queries_set_name='SOS Queries')


# Run the queries using RNSB
RNSB_gender_results = run_queries(RNSB,
                                gender_queries,
                                models,
                                aggregate_results=True,
                                return_only_aggregation=True,
                                    lost_vocabulary_threshold = 0.6,
                                queries_set_name='SOS Queries')




In [18]:
# Run the queries using RND
RND_gender_results = run_queries(RND,
                                gender_queries,
                                models,
                                aggregate_results=True,
                                generate_subqueries=True,
                                lost_vocabulary_threshold = 0.6,
                                queries_set_name='SOS Queries')




In [19]:
# Run the queries using ECT
ECT_gender_results = run_queries(ECT,
                                gender_queries,
                                models,
                                aggregate_results=True,
                                generate_subqueries=True,
                                lost_vocabulary_threshold = 0.6,
                                queries_set_name='SOS Queries')

In [20]:
WEAT_gender_results

,WEAT: SOS Queries average of abs values score
model_name,
fasttext_wiki,0.061530
fasttext_wiki_subwords,0.044221
fasttext_cc,0.019011
fasttext_cc_subwords,0.024218
debiased_w2v,0.003613


In [21]:
RNSB_gender_results

,RNSB: SOS Queries average of abs values score
model_name,
fasttext_wiki,0.105471
fasttext_wiki_subwords,0.005187
fasttext_cc,0.191069
fasttext_cc_subwords,0.082303
debiased_w2v,0.059441


In [22]:
RND_gender_results["RND: SOS Queries average of abs values score"]

model_name
fasttext_wiki             0.022077
fasttext_wiki_subwords    0.033635
fasttext_cc               0.024758
fasttext_cc_subwords      0.060290
debiased_w2v              0.009826
Name: RND: SOS Queries average of abs values score, dtype: float64

In [23]:
ECT_gender_results["ECT: SOS Queries average of abs values score"]

model_name
fasttext_wiki             0.924477
fasttext_wiki_subwords    0.948985
fasttext_cc               0.847977
fasttext_cc_subwords      0.935170
debiased_w2v              0.986154
Name: ECT: SOS Queries average of abs values score, dtype: float64

In [24]:
import numpy as np
import scipy

In [25]:
ECT_gender_results["ECT: SOS Queries average of abs values score"]

model_name
fasttext_wiki             0.924477
fasttext_wiki_subwords    0.948985
fasttext_cc               0.847977
fasttext_cc_subwords      0.935170
debiased_w2v              0.986154
Name: ECT: SOS Queries average of abs values score, dtype: float64

In [26]:
ECT_gender_results

,Target set 0 and Target set 1 wrt Attribute set 0,Target set 0 and Target set 1 wrt Attribute set 1,ECT: SOS Queries average of abs values score
model_name,,,
fasttext_wiki,0.885257,0.963697,0.924477
fasttext_wiki_subwords,0.930289,0.967682,0.948985
fasttext_cc,0.773962,0.921991,0.847977
fasttext_cc_subwords,0.903460,0.966879,0.935170
debiased_w2v,0.975535,0.996773,0.986154


In [27]:
gender_ranking = create_ranking([ WEAT_gender_results, 
                                RNSB_gender_results, RND_gender_results
                                ], ascending=True)
ect_gender_Ranking = create_ranking([ECT_gender_results], ascending=False)

In [28]:
ect_gender_Ranking

,ECT: SOS Queries average of abs values score
model_name,
fasttext_wiki,4.0
fasttext_wiki_subwords,2.0
fasttext_cc,5.0
fasttext_cc_subwords,3.0
debiased_w2v,1.0


In [29]:
gender_ranking["ECT"] = ect_gender_Ranking["ECT: SOS Queries average of abs values score"].values

In [30]:
gender_ranking

,WEAT: SOS Queries average of abs values score,RNSB: SOS Queries average of abs values score,RND: SOS Queries average of abs values score,ECT
model_name,,,,
fasttext_wiki,5.0,4.0,2.0,4.0
fasttext_wiki_subwords,4.0,1.0,4.0,2.0
fasttext_cc,2.0,5.0,3.0,5.0
fasttext_cc_subwords,3.0,3.0,5.0,3.0
debiased_w2v,1.0,2.0,1.0,1.0
